<a href="https://colab.research.google.com/github/Bikas0/NLP-Text-Summarizer/blob/main/NLP_Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**importing libraries**

In [1]:
import re
import nltk
import spacy
import string
import numpy as np
import pandas as pd

**Downloading required files (download if needed)**

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
!pip install wikipedia
import wikipedia

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=dc9176331305982690284e9623924497d7c84fc1d5a19c83a890951c3539a3ea
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [6]:
# Input text - to summarize  
# text = open("Elon Musk.txt",'r',encoding='utf8')
# text=text.read()
# text
 
# from Wikipedia
Albert_Einstein = wikipedia.page('Albert_Einstein')
text = Albert_Einstein.content

In [7]:
text

'Albert Einstein ( EYEN-styne; German: [ˈalbɛʁt ˈʔaɪnʃtaɪn] (listen); 14 March 1879 – 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. Einstein is best known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics. Relativity and quantum mechanics are together the two pillars of modern physics. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been dubbed "the world\'s most famous equation". His work is also known for its influence on the philosophy of science. He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory. His intellectual achievements and originality resulted in "Einstein" becoming synonymous with "genius".In 1

In [8]:
len(text)

76071

In [9]:
# removing waste elements
text = re.sub('\[[a-zA-z0-9\s]+\]', '', text) # removing character like [34 text] etc
text = re.sub('=+\s+\d?-?[a-zA-z0-9\:–\s-]+\s+=+', '', text) # removing character like == see also ==
 
exclist = string.punctuation
# remove punctuations and digits from oldtext
table_ = str.maketrans('', '', exclist)
newtext = text.translate(table_).lower()

In [10]:
newtext

'albert einstein  eyenstyne german ˈalbɛʁt ˈʔaɪnʃtaɪn listen 14 march 1879 – 18 april 1955 was a germanborn theoretical physicist widely acknowledged to be one of the greatest and most influential physicists of all time einstein is best known for developing the theory of relativity but he also made important contributions to the development of the theory of quantum mechanics relativity and quantum mechanics are together the two pillars of modern physics his mass–energy equivalence formula e  mc2 which arises from relativity theory has been dubbed the worlds most famous equation his work is also known for its influence on the philosophy of science he received the 1921 nobel prize in physics for his services to theoretical physics and especially for his discovery of the law of the photoelectric effect a pivotal step in the development of quantum theory his intellectual achievements and originality resulted in einstein becoming synonymous with geniusin 1905 a year sometimes described as h

In [11]:
len(newtext)

72298

In [31]:
#Frequency table

def freqTab(text, stopword):
  wrd_tkn = nltk.tokenize.word_tokenize(text)
  without_stopwords = [x for x in wrd_tkn if x not in stopword]
  #word frequency
  word_dict = {}
  for i in without_stopwords:
    if i in word_dict:
      word_dict[i] += 1
    else:
      word_dict[i] = 1
  return word_dict      


In [36]:
stpWrd = nltk.corpus.stopwords.words('english')
word_count = freqTab(newtext, stpWrd)
FreqTable = pd.DataFrame({
    'Key': list(word_count.keys()),
    'Value': list(word_count.values())
})

FreqTable = FreqTable.sort_values(by = 'Value', ascending = False)
FreqTable

,Key,Value
1,einstein,214
27,theory,54
273,einsteins,42
28,relativity,42
40,physics,33
...,...,...
1395,hanged,1
1394,phrase,1
1392,enemies,1
1391,list,1


In [33]:
# creating sentence tokenizer
sent_token = nltk.tokenize.sent_tokenize(text) #using original text to keep punctuation marks

In [37]:
FreqTable

,Key,Value
1,einstein,214
27,theory,54
273,einsteins,42
28,relativity,42
40,physics,33
...,...,...
1395,hanged,1
1394,phrase,1
1392,enemies,1
1391,list,1


In [40]:
def sent_weight(sentence, word_count):
  sent_rank = {}
  for sent in sentence:
    for word, freq in word_count.items():
      if word in sent.lower():
        if sent in sent_rank:
          sent_rank[sent] += freq
        else:
          sent_rank[sent] = freq
      else:
        pass 
  return sent_rank
sent_rank = sent_weight(sent_token, word_count)              

In [41]:
sent_token

['Albert Einstein ( EYEN-styne; German: [ˈalbɛʁt ˈʔaɪnʃtaɪn] (listen); 14 March 1879 – 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time.',
 'Einstein is best known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics.',
 'Relativity and quantum mechanics are together the two pillars of modern physics.',
 'His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been dubbed "the world\'s most famous equation".',
 'His work is also known for its influence on the philosophy of science.',
 'He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory.',
 'His intellectual achievements and originality resulted in "Einstein" becoming syno

In [42]:
sent_rank

{'"Although Einstein was wrong about local realism, his clear prediction of the unusual properties of its opposite, entangled quantum states, has resulted in the EPR paper becoming among the most influential papers published in Physical Review.': 432,
 '"Einstein bequeathed his personal archives, library, and intellectual assets to the Hebrew University of Jerusalem in Israel.': 328,
 '"Einstein has been the subject of or inspiration for many novels, films, plays, and works of music.': 306,
 '"Einstein was primarily affiliated with non-religious humanist and Ethical Culture groups in both the UK and US.': 272,
 '"In 1922, his travels took him to Asia and later to Palestine, as part of a six-month excursion and speaking tour, as he visited Singapore, Ceylon and Japan, where he gave a series of lectures to thousands of Japanese.': 157,
 '"Music took on a pivotal and permanent role in Einstein\'s life from that period on.': 292,
 '... For me the Jewish religion like all other religions is

In [43]:
#Let's see the what is the average score achieved
average = sum(sent_rank.values())/len(sent_rank)
print(f'Total sentence: {len(sent_rank)} \nTotal Score: {sum(sent_rank.values())} \nAverage Score: {average}',)

Total sentence: 453 
Total Score: 109145 
Average Score: 240.93818984547462


In [44]:
# let's summarize it 
# Storing sentences into our summary. 

summary = ''
counter = 0
for sentence in sent_rank:
  if (sentence in sent_token) and (sent_rank[sentence] > (2.2*average)):
    summary += " " + sentence
    counter += 1

summary

' There is eyewitness evidence and several letters over many years that indicate Marić might have collaborated with Einstein prior to his landmark 1905 papers, known as the Annus Mirabilis papers, and that they developed some of the concepts together during their studies, although some historians of physics who have studied the issue disagree that she made any substantive contributions. His thesis was accepted in July 1905, and Einstein was awarded a PhD on January 15, 1906.Also in 1905, which has been called Einstein\'s annus mirabilis (amazing year), he published four groundbreaking papers, on the photoelectric effect, Brownian motion, special relativity, and the equivalence of mass and energy, which were to bring him to the notice of the academic world, at the age of 26. He also studied continuum mechanics, the molecular theory of heat, and the problem of gravitation, on which he worked with mathematician and friend Marcel Grossmann.When the "Manifesto of the Ninety-Three" was publi

In [45]:
len(summary)

3814

In [46]:
# import gensim
from gensim.summarization import summarize
summarize(text,ratio=0.15)

'He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory.\nThese outlined the theory of the photoelectric effect, explained Brownian motion, introduced special relativity, and demonstrated mass-energy equivalence.\nEinstein thought that the laws of classical mechanics could no longer be reconciled with those of the electromagnetic field, which led him to develop his special theory of relativity.\nHe then extended the theory to gravitational fields; he published a paper on general relativity in 1916, introducing his theory of gravitation.\nIn 1917, he applied the general theory of relativity to model the structure of the universe.\nAt the age of eight, he was transferred to the Luitpold-Gymnasium (now known as the Albert-Einstein-Gymnasium), where he received advanced primary and secondary school education until he left the German Em